In [1]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from amftrack.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number

import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment
from amftrack.pipeline.paths.directory import run_parallel, find_state, directory_scratch, directory_project, path_code

from amftrack.notebooks.analysis.data_info import *
import matplotlib.patches as mpatches
from statsmodels.stats import weightstats as stests


In [3]:
inst_25 = [(9,0,11),(9,13,35),(3,0,19),(37,0,8),(11,0,30),(19,0,25),(13,0,25),(39,0,18),(35,0,15),(29,0,20)]
inst_bait = [(10,0,10), (14,0,11), (33,0,26),(4,2,18),(4,20,30),(39,117,137),(12,5,21),(28,0,14),(32,5,14),(32,15,44),(36,0,9),(40,0,14),(2,1,15),(5,160,168),(11,158,164),(13,116,131)]
inst_30 = []
inst_25late = [(40,153,153+37)]
treatments = {'25*' : inst_25late,'25' : inst_25,'baits' : inst_bait, '30' : inst_30}
plate_number = {(9,0,11) : 296, (9,13,35) : 296, (3,0,19) : 340, (37,0,8) : 269,(11,0,30) : 314, (19,0,25) : 344, (13,0,25) : 298, (39,0,18) : 297, (35,0,15) : 351,(10,0,10) : 395,(14,0,11) : 399, (33,0,26) : 420, (4,2,18) : 423, (4,20,30) : 423,(8,0,17): 434 ,(8,20,30) : 434,(39,117,137) : 433, (12,5,21) : 436, (28,0,14): 405,(32,5,45):409,(36,0,9) : 419,(40,0,14) : 425,(2,1,15):435,(2,16,35):435,(5,160,168):382,(11,158,164) : 416,(13,116,131) : 424, (29,0,20) : 373,(32,15,44):409, (32,5,14) : 409, (40,153,153+37) : 69}

In [2]:
window=800
results={}
for treatment in treatments.keys():
    insts = treatments[treatment]
    for inst in insts:
        results[inst] = pickle.load(open(f'{path_code}/MscThesis/Results/straight_{window}_{inst}.pick', "rb"))

***tortuosities vs treatment***

In [4]:
column_names = ["plate","inst", "treatment", "angle", "curvature","density","growth","speed","straightness","t","hyph"]
infos = pd.DataFrame(columns=column_names)
for treatment in treatments.keys():
    insts = treatments[treatment]
    for inst in insts:
        angles, curvatures, densities,growths,speeds,tortuosities,ts,hyphs = results[inst]
        for i,angle in enumerate(angles):
            new_line = pd.DataFrame(
                {   "plate": [plate_number[inst]],
                    "inst": [inst],
                    "treatment": [treatment],
                    "angle": [angle],
                    "curvature": [curvatures[i]],
                    "density": [densities[i]],
                    "growth": [growths[i]],
                    "speed": [speeds[i]],
                    "straightness": [tortuosities[i]],
                     "t": [ts[i]],
                     "hyph": [hyphs[i]],
                }
            )  # index 0 for
            # mothers need to be modified to resolve multi mother issue
            infos = infos.append(new_line, ignore_index=True)

ValueError: too many values to unpack (expected 8)

In [4]:
infos.to_csv(f'{path_code}/MscThesis/Results/straight_{window}.csv')

In [3]:
infos

NameError: name 'infos' is not defined

In [1]:
corrected = infos.loc[(infos["straightness"] <= 1)&(infos["plate"] !=435)& (infos["speed"] >=25)]

NameError: name 'infos' is not defined

In [5]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
bplot1 = corrected.boxplot(column = ['curvature'],by="treatment",figsize =(9,8),ax =ax,patch_artist=True, showfliers=False, notch=True,showmeans = True)
colors = 1*['lightblue']+ 16*['pink']
for i,(artist, col) in enumerate(zip(ax.artists, colors)):
    artist.set_edgecolor(col)
    artist.set_facecolor(col) 
ax.set_xlabel('Plate')
ax.set_ylabel('handedness (°.um-1)')
# ax.set_ylim(0.9)
plt.show()

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [9]:
def set_axis_style(ax, labels):
    ax.xaxis.set_tick_params(direction='out')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_xticks(np.arange(1, len(labels) + 1))
    ax.set_xticklabels(labels)
    ax.set_xlim(0.25, len(labels) + 0.75)
    ax.set_xlabel('Treatment')

In [11]:
plt.close('all')
fig, axes = plt.subplots()
value = 'curvature'
axes.violinplot(dataset = [corrected[corrected.treatment == '25'][value].values,
                           corrected[corrected.treatment == 'baits'][value].values,], showmeans = True)

axes.set_ylabel('Handedness(°.um-1)')
labels = ['soluble','insoluble']
set_axis_style(axes, labels)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1]:
fig = plt.figure()
bins = np.linspace(0, 400, 50)
ax = fig.add_subplot(111)
ax.hist(corrected.loc[corrected['treatment'] == '25']['speed'],bins,alpha=0.3,label='Dummy baits (homogeneous soluble)',density=True)
ax.hist(corrected.loc[corrected['treatment'] == 'baits']['speed'],bins,alpha=0.3,label='P&N baits (heterogeneous rock form)',density=True)
ax.hist(corrected.loc[corrected['treatment'] == '25*']['speed'],bins,alpha=0.3,label='No baits (homogeneous soluble)',density=True)
ax.set_xlabel(r'speed ($\mu m.h^{-1} $)')
plt.legend(loc='upper right')
ax.set_ylabel(r'density')


NameError: name 'plt' is not defined

In [7]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
bplot1 = corrected.boxplot(column = ['speed'],by="plate",figsize =(9,8),ax =ax,patch_artist=True, showfliers=False,notch=True,showmeans = True)
colors = 3*['lightgreen'] + 9 *['lightblue'] + 16*['pink']
for i,(artist, col) in enumerate(zip(ax.artists, colors)):
    artist.set_edgecolor(col)
    artist.set_facecolor(col) 
ax.set_xlabel('Plate')
ax.set_ylabel(r'Speed($\mu m.h^{-1})$')
# labels = ['dummy baits', 'no baits', 'P&N baits']
# set_axis_style(ax, labels)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [11]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
bplot1 = corrected.boxplot(column = ['speed'],by="treatment",figsize =(9,8),ax =ax,patch_artist=True, showfliers=False,notch=True,showmeans = True)
colors =  ['lightblue']  + ['lightgreen'] + ['pink']
for i,(artist, col) in enumerate(zip(ax.artists, colors)):
    artist.set_edgecolor(col)
    artist.set_facecolor(col) 
ax.set_xlabel('Plate')
ax.set_ylabel(r'Speed($\mu m.h^{-1})$')
labels = ['dummy baits', 'no baits', 'P&N baits']
set_axis_style(ax, labels)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [52]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
bplot1 = corrected.boxplot(column = ['straightness'],by="plate",figsize =(9,8),ax =ax,patch_artist=True, showfliers=False,notch=True,showmeans = True,bootstrap =10000)
colors = ['lightgreen']+9*['lightblue']+ +16*['pink']
for i,(artist, col) in enumerate(zip(ax.artists, colors)):
    artist.set_edgecolor(col)
    artist.set_facecolor(col) 
ax.set_xlabel('Treatment')
ax.set_ylabel('Straightness')
ax.set_ylim(0.95)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
bplot1 = corrected.boxplot(column = ['straightness'],by="treatment",figsize =(9,8),ax =ax,patch_artist=True, showfliers=False,notch=True,showmeans = True,bootstrap =10000)
colors = ['lightblue']+ ['lightgreen']+['pink']
for i,(artist, col) in enumerate(zip(ax.artists, colors)):
    artist.set_edgecolor(col)
    artist.set_facecolor(col) 
ax.set_xlabel('Treatment')
ax.set_ylabel('Straightness')
ax.set_ylim(0.95)
labels = ['dummy baits', 'no baits', 'P&N baits']
ax.set_xticks(np.arange(1, len(labels) + 1))
ax.set_xticklabels(labels)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
angles_25 = corrected.loc[infos['treatment']=='25']['straightness']
angles_baits = corrected.loc[infos['treatment']=='baits']['straightness']
angles_no_baits = corrected.loc[infos['treatment']=='25*']['straightness']

fig = plt.figure()
bins = np.linspace(0.98, 1, 30)
ax = fig.add_subplot(111)
ax.hist(angles_25,bins,alpha=0.3,label='dummy baits',density=True)
ax.hist(angles_baits,bins,alpha=0.3,label='P&N baits',density=True)
# ax.hist(angles_no_baits,bins,alpha=0.3,label='no baits',density=True)

ax.set_xlabel(r'Straightness')
plt.legend(loc='upper left')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
ax.scatter(corrected['growth'],corrected['straightness'])
ax.set_xlim(400,1500)
ax.set_ylim(0.97,1.01)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.97, 1.01)

In [9]:
len(densities_sort)

5370

In [13]:
from scipy import stats
from scipy.ndimage import uniform_filter1d
zipped_list = zip(corrected['growth'],corrected['straightness'])
sorted_zip = sorted(zipped_list)
densities_sort,curvatures_sort = zip(*sorted_zip)
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('curvilinear length ($\mu m$')
ax.set_ylabel('straightness ($\mu m. \mu m^{-1}$)')
xlim = 1500
ax.set_xlim(400,1500)
ax.set_ylim((0.97,1.01))
slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort[:4500],np.abs(curvatures_sort)[:4500])
x= range(400,xlim)

N=1000
ax.scatter(densities_sort,curvatures_sort,color='yellow',label='observed speeds')
ax.plot(uniform_filter1d(densities_sort, size=N),uniform_filter1d(curvatures_sort, size=N),color='green',label = 'moving average')
# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
x= range(0,xlim+1)
legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
corrected.loc[(corrected["treatment"] == "25")]["speed"]

0       143.732337
1       137.746208
2       157.329867
3       189.751303
4       185.732163
           ...    
2417    149.133300
2418    137.772812
2419    123.450261
2420    118.732283
2421    101.654672
Name: speed, Length: 2289, dtype: float64

In [18]:
ztest ,pval1 = stests.ztest(corrected.loc[(corrected["treatment"] == "25")]["speed"],corrected.loc[corrected["treatment"] == "baits"]["speed"], value=0,alternative='two-sided')
pval1,ztest

(0.0, -41.02315358212294)

In [20]:
ztest ,pval1 = stests.ztest(corrected.loc[(corrected["treatment"] == "25")]['straightness'],corrected.loc[corrected["treatment"] == "baits"]['straightness'], value=0,alternative='two-sided')
pval1,ztest

(3.2407949683100726e-12, 6.966865816438617)

In [7]:
corrected.boxplot(column = ['straightness'],by="treatment",figsize =(9,8))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
fitted_data,fitted_lambda = stats.boxcox(1-corrected.loc[corrected["treatment"] == 'baits']['straightness']) 

In [55]:
fitted_lambda 

0.053559026311838766

In [11]:
data25 = stats.boxcox(1-corrected.loc[corrected["treatment"] == '25']['straightness'],lmbda = fitted_lambda) 
data30 = stats.boxcox(1-corrected.loc[corrected["treatment"] == '30']['straightness'],lmbda = fitted_lambda) 
databaits = stats.boxcox(1-corrected.loc[corrected["treatment"] == 'baits']['straightness'],lmbda = fitted_lambda) 

NameError: name 'stats' is not defined

In [10]:
import statsmodels.formula.api as sm
sm.ols('straightness ~ density + speed',data = corrected.loc[corrected["treatment"] == '25']).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           straightness   R-squared:                       0.075
Model:                            OLS   Adj. R-squared:                  0.074
Method:                 Least Squares   F-statistic:                     92.51
Date:                Wed, 07 Apr 2021   Prob (F-statistic):           2.32e-39
Time:                        16:14:57   Log-Likelihood:                 5953.4
No. Observations:                2289   AIC:                        -1.190e+04
Df Residuals:                    2286   BIC:                        -1.188e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.9936      0.002    592.558      0.000       0.990       0.997
density       -0.0028      0.000    -13.570      0.000      -0.003      -0.002
speed      -2.098e-06   9.76e-06     -0.215      0.830   -2.12e-05     1.7e-05
==============================================================================
Omnibus:                     2024.672   Durbin-Watson:                   1.981
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            71082.003
Skew:                          -4.128   Prob(JB):                         0.00
Kurtosis:                      29.022   Cond. No.                         714.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
sm.ols('straightness ~ density + speed',data = corrected.loc[corrected["plate"] == 69]).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           straightness   R-squared:                       0.100
Model:                            OLS   Adj. R-squared:                  0.096
Method:                 Least Squares   F-statistic:                     26.91
Date:                Fri, 29 Jan 2021   Prob (F-statistic):           8.31e-12
Time:                        12:16:41   Log-Likelihood:                 1220.5
No. Observations:                 487   AIC:                            -2435.
Df Residuals:                     484   BIC:                            -2422.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.9606      0.005    198.905      0.000       0.951       0.970
density       -0.0002      0.001     -0.322      0.748      -0.001       0.001
speed          0.0001   1.77e-05      6.454      0.000    7.94e-05       0.000
==============================================================================
Omnibus:                      346.831   Durbin-Watson:                   1.559
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3940.374
Skew:                          -3.081   Prob(JB):                         0.00
Kurtosis:                      15.499   Cond. No.                     1.24e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.24e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [20]:
1000/1.725

579.7101449275362

In [54]:
ztest ,pval1 = stests.ztest(data25,data30, value=0,alternative='two-sided')
pval1

3.952093842738811e-33

In [52]:
ztest ,pval1 = stests.ztest(data25,databaits, value=0,alternative='two-sided')
pval1

1.5692013454492557e-10

In [48]:
ztest ,pval1 = stests.ztest(data25,data30,corrected.loc[corrected["inst"] == (22,8,26)]['tortuosity'], corrected.loc[corrected["inst"] == (26,30,53)]['tortuosity'], value=0,alternative='two-sided')
pval1

NameError: name 'stests' is not defined

In [66]:
corrected = infos.loc[infos["speed"] >= 100]
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
bplot1 = corrected.boxplot(column = ["speed"],by="plate",figsize =(9,8),ax =ax,patch_artist=True)
colors = ['pink']*3+['lightblue']*3 +['lightgreen']*3
for i,(artist, col) in enumerate(zip(ax.artists, colors)):
    artist.set_edgecolor(col)
    artist.set_facecolor(col) 
ax.set_xlabel('Plate number')
ax.set_ylabel('Straightness')
plt.show()

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
window=800
for treatment in treatments.keys():
    insts = treatments[treatment]
    for inst in insts:
        result = get_curvature_density(inst,window)
        pickle.dump(result, open(f'{path_code}/MscThesis/Results/straight_{window}_{inst}.pick', "wb"))

begin = 2020-07-01 15:57:00 
  end = 2020-07-04 19:57:00
There is 148 RH
begin = 2020-11-23 15:01:00 
  end = 2020-11-26 03:02:00


In [42]:
angles,curvatures = get_all_curvature((24,104,116))

begin = 2020-12-08 10:57:00 
  end = 2020-12-10 11:07:00


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

There is 531 RH


In [43]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.set_xlabel('angle(°)')
ax.set_ylabel('density')
# ax.set_xlim((0,2))
ax.set_title(f'mean = {round(np.nanmean(angles),max(2,-int(np.log10(abs(np.nanmean(angles))))+2))}, std = {round(np.nanstd(angles),max(2,-int(np.log10(np.nanstd(angles)))+2))}')
ax.hist(angles,10,density=True)
np.nanmean(angles),np.nanstd(angles)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-4.00015425895818, 53.04022492898531)

In [45]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.set_xlabel('"curvature",°/um')
ax.set_ylabel('density')
# ax.set_xlim((0,2))
ax.set_title(f'mean = {round(np.nanmean(curvatures),max(2,-int(np.log10(abs(np.nanmean(curvatures))))+2))}, std = {round(np.nanstd(curvatures),max(2,-int(np.log10(np.nanstd(curvatures)))+2))}')

ax.hist(curvatures,10,density=True)
np.nanmean(curvatures),np.nanstd(curvatures)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-0.0056825003334708595, 0.05425367450462862)

In [67]:
def get_curvature_density(inst,window):
    exp = get_exp(inst)
    skeletons = [sparse.csr_matrix(skel) for skel in exp.skeletons]
    RH, BAS, max_growths, total_growths, lengths, branch_frequ, select_hyph = get_rh_bas(exp,criter)
    angles = []
    curvatures = []
    densities = []
    growths = []
    speeds = []
    tortuosities = []
    print(f'There is {len(RH)} RH')
    for hyph in RH:
        for i,t in enumerate(hyph.ts[:-1]):
            tp1=hyph.ts[i+1]
            segs,nodes = get_pixel_growth_and_new_children(hyph,t,tp1)
            speed = np.sum([get_length_um(seg) for seg in segs])/get_time(exp,t,tp1)
            total_growth = speed * get_time(exp,t,tp1)
            curve = [pixel for seg in segs for pixel in seg]
            pos = hyph.end.pos(t)
            x,y = pos[0],pos[1]
            straight_distance = np.linalg.norm(hyph.end.pos(t)-hyph.end.pos(tp1))
            skeleton=skeletons[t][x-window:x+window,y-window:y+window]
            density = skeleton.count_nonzero()/(window*1.725)
            curve_array = np.array(curve)
            res = 50
            index = min(res,curve_array.shape[0]-1)
            vec1 = curve_array [index]-curve_array [0]
            vec2 = curve_array [-1]-curve_array [-index-1]
            if np.linalg.norm(vec1)>0 and np.linalg.norm(vec2)>0:
                begin=vec1/np.linalg.norm(vec1)
                end=vec2/np.linalg.norm(vec2)
                dot_product = min(np.dot(begin, end),1)
                if (begin[1] * end[0] - end[1] * begin[0] >= 0):  # determinant
                    angle = -np.arccos(dot_product) / (2 * np.pi) * 360
                else:
                    angle = np.arccos(dot_product) / (2 * np.pi) * 360
                inv_tortuosity = (straight_distance*1.725)/total_growth
                if  speed>=100 and speed <400 and hyph.end.degree(tp1)<3 and inv_tortuosity>0.8 and inv_tortuosity<1.1:
                    if np.isnan((angle/total_growth)):
                        print(angle,total_growth,dot_product)
                    angles.append(angle)
                    curvatures.append(angle/total_growth)
                    densities.append(density)
                    growths.append(total_growth)
                    speeds.append(speed)
                    tortuosities.append(inv_tortuosity)
    return(angles, curvatures, densities,growths,speeds,tortuosities)

In [68]:
angles,curvatures, densities,growths, speeds, tortuosities = get_curvature_density((40,0,37),800)

begin = 2020-11-19 20:21:00 
  end = 2020-11-25 18:06:00
There is 189 RH


In [85]:
plt.close('all')
zipped_list = zip(speeds,np.abs(tortuosities))
sorted_zip = sorted(zipped_list)
densities_sort,curvatures_sort = zip(*sorted_zip)
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('speed ($\mu m.hour^{-1})$')
ax.set_ylabel('straightness ($\mu m. \mu m^{-1}$)')
xlim = int(np.max(densities_sort))+1
ax.set_xlim((0,xlim))
ax.set_ylim((0.8,1.05))
slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(0,xlim)

N=30
ax.scatter(densities_sort,curvatures_sort,color='yellow',label='observed speeds')
ax.plot(uniform_filter1d(densities_sort, size=N),uniform_filter1d(curvatures_sort, size=N),color='green',label = 'moving average')
# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
x= range(0,xlim+1)
legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [89]:
plt.close('all')
zipped_list = zip(densities,np.abs(tortuosities))
sorted_zip = sorted(zipped_list)
densities_sort,curvatures_sort = zip(*sorted_zip)
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('density ($\mu m. \mu m^{-1}$)')
ax.set_ylabel('straightness ($\mu m. \mu m^{-1}$)')
xlim = int(np.max(densities_sort))+1
ax.set_xlim((0,xlim))
ax.set_ylim((0.8,1.05))
slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(0,xlim)

N=30
ax.scatter(densities_sort,curvatures_sort,color='yellow',label='observed speeds')
ax.plot(uniform_filter1d(densities_sort, size=N),uniform_filter1d(curvatures_sort, size=N),color='green',label = 'moving average')
# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
x= range(0,xlim+1)
legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
zipped_list = zip(speeds,np.abs(tortuosities))
sorted_zip = sorted(zipped_list)
densities_sort,curvatures_sort = zip(*sorted_zip)
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
# ax.set_xlabel('time since start of imaging ($hour$)')
# ax.set_ylabel('speed ($\mu m.hour^{-1})$')
xlim = 10
# ax.set_xlim((0,xlim))
ax.set_ylim((0,3))
slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(0,xlim)

N=30
ax.scatter(densities_sort,curvatures_sort,color='yellow',label='observed speeds')
ax.plot(uniform_filter1d(densities_sort, size=N),uniform_filter1d(curvatures_sort, size=N),color='green',label = 'moving average')
# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
x= range(0,xlim+1)
legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}'
line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)
ax.legend()

In [12]:

slope, intercept, r_value, p_value, std_err = stats.linregress(curvatures,np.abs(curvatures))

In [13]:
p_value

7.875893889324231e-13

In [14]:
slope

-0.0004887144099304017

In [ ]:
exp_inst = [(22,8,40)]

experiments = []
for inst in exp_inst:
    plate = inst[0]
    begin = inst[1]
    end = inst[2]
    dates_datetime = get_dates_datetime(directory,plate)
    print('begin =',dates_datetime[begin],'\n  end =',dates_datetime[end])
    dates_datetime_chosen=dates_datetime[begin:end+1]
    dates = dates_datetime_chosen
    exp = pickle.load( open(f'{directory}/Analysis_Plate{plate}_{dates[0]}_{dates[-1]}/experiment_{plate}.pick', "rb" ) )
    experiments.append(exp)

In [ ]:
select_hyph = {}
for hyph in exp.hyphaes:
    select_hyph[hyph] = []
    for i,t in enumerate(hyph.ts[:-1]):
        tp1=hyph.ts[i+1]
        pixels,nodes = get_pixel_growth_and_new_children(hyph,t,tp1)
        length = np.sum([get_length_um(seg) for seg in pixels])/get_time(exp,t,tp1)
        select_hyph[hyph].append((t,hyph.ts[i+1],length,pixels))
        

In [ ]:
max_growths = []
total_growths = []
lengths = []
branch_frequ = []
hyph_l = []
RH=[]
BAS=[]
thresh_growth = 100
thresh_length = 2000
for hyph in exp.hyphaes:
    growths = [c[2] for c in select_hyph[hyph]]
    ts = [c[0] for c in select_hyph[hyph]]
    tp1s = [c[1] for c in select_hyph[hyph]]
    if len(growths)>0:
        length = hyph.get_length_um(hyph.ts[-1])
        nodes = hyph.get_nodes_within(hyph.ts[-1])[0]
        max_growth = np.max(growths)
        total_growth = np.sum([growth * get_time(exp,ts[i],tp1s[i]) for i,growth in enumerate(growths)])
        if max_growth>=thresh_growth and length>=thresh_length:
            RH.append(hyph)
        else:
            BAS.append(hyph)
        lengths.append(length)
        max_growths.append(max_growth)
        total_growths.append(total_growth)
        branch_frequ.append((len(nodes)-1)/(length+1))
        hyph_l.append(hyph)

In [ ]:
rh = choice(RH)
patterns = select_hyph[rh]
ts = [c[0] for c in select_hyph[rh]]
tp1s = [c[1] for c in select_hyph[rh]]
growths = [c[2] for c in select_hyph[rh]]
if len(growths)==0:
    print('error')
total_growth = np.sum([growth * get_time(exp,ts[i],tp1s[i]) for i,growth in enumerate(growths)])
curve = [pixel for info in patterns for seg in info[3] for pixel in seg]

In [ ]:
curve_array = np.array(curve)

In [ ]:
res = 50
vec1 = curve_array [res]-curve_array [0]
vec2 = curve_array [-1]-curve_array [-res-1]

In [90]:
begin=vec1/np.linalg.norm(vec1)
end=vec2/np.linalg.norm(vec2)

In [91]:
plt.close('all')
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
# ax.set_xlabel('density(um hypha/um^2)')
# ax.set_ylabel('speed (um/hour)')
# # ax.set_xlim((0,15000))
# ax.set_ylim((0,450))
ax.scatter(curve_array[:,1],curve_array[:,0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [92]:
dot_product = np.dot(begin, end)
if (begin[1] * end[0] - end[1] * begin[0] >= 0):  # determinant
    angle = -np.arccos(dot_product) / (2 * np.pi) * 360
else:
    angle = np.arccos(dot_product) / (2 * np.pi) * 360

In [93]:
angle/total_growth

-0.004606482394418124

In [94]:
angle

-11.479949124900518

In [95]:
plt.close('all')
exp.plot(rh.ts,[[rh.end.label,rh.root.label]]*len(rh.ts))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
len(angles)

18